<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/getting-started-with-nlp/03-information-retrieval/information_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Information Retrieval

You might have come across the term information retrieval in the context of
search engines; for example, Google famously started its business by providing a
powerful search algorithm that kept improving over time. The search for information,
however, is a basic need that you may face beyond searching online.

For
instance, every time you search for the files on your computer, you are performing
a sort of information retrieval.

Information search is based on
the idea that the content of a document or set of documents is relevant given the content
of a particular query, so a documents data structure should keep the contents of
all available documents for the algorithm to select from.



##Setup

In [ ]:
import math
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize, WordNetLemmatizer, pos_tag
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from operator import itemgetter

##Dataset

There are three components to this data:

* documents with their ids and content-there are 1460 of those to be precise
* questions/queries with their ids and content- there are 112 of those;
* mapping between the queries and relevant documents

First, let's read in documents from the `CISI.ALL` file and store the result in `documents` data structure- set of tuples of document ids matched with contents:

In [ ]:
# populate the documents dictionary